### Imports 📚

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


### Scrape Data 🌐

In [2]:
# Select cities to get information about
cities = ['Paphos', 'Rehovot']
htmls = []

#Iterate over cities and scrape data
for city in cities:
    url = f'https://www.numbeo.com/cost-of-living/in/{city}?displayCurrency=EUR'
    data = requests.get(url)

    with open(f'{city}_numbeo.html','w+') as file:
        file.write(data.text)
        htmls.append(file.name)



### Turn to Pandas DataFrames 🐼 & Clean 🧼

In [3]:
df_names = []

for html in htmls:
    # Use beautifulsoup to parse the html file
    soup = BeautifulSoup(open(html),'html.parser')

    # Create a variable to hold only the wanted table
    table = soup.find('table', class_='data_wide_table new_bar_table')

    # Search for the categories
    category_divs = soup.find_all('div', class_='category_title')
    categories = [div.text.strip() for div in category_divs]

    # Turn into a DataFrame
    df = pd.read_html(str(table))[0]

    # Save current category to use in first df rows
    current_category = categories.pop(0)
    df['categories'] = current_category

    # Iterate over the df and assign correct category
    for index, row in df.iterrows():
        if row['Edit'] == 'Edit':
            # Assign the next category from the list
            current_category = categories.pop(0) 
        # Assign the current category to the row
        df.at[index, 'categories'] = current_category  

    # Remove the rows with 'Edit' in the 'Edit' column
    df = df[df['Edit'] != 'Edit']  

    # Rename the Columns
    df = df.rename(columns={'Restaurants': 'Item', 'Edit': 'Average_Price_in_Eur', 'categories': 'Category'})

    # Turn average price to numeric and remove currency notation
    df['Average_Price_in_Eur'] = pd.to_numeric(df['Average_Price_in_Eur'].str.replace('[^\d.]', ''), errors='coerce')

    # Remove commas from the Range column
    df['Range'] = df['Range'].str.replace(',', '')

    # Add a minimum and maximum columns from range column
    df[['Minimum', 'Maximum']] = df['Range'].str.split('-', expand=True)
    df['Minimum'] = pd.to_numeric(df['Minimum'], errors='coerce')
    df['Maximum'] = pd.to_numeric(df['Maximum'], errors='coerce')

    # Remove rows with NaNs in Range or Average Price
    df = df.dropna(subset=['Range', 'Average_Price_in_Eur'])

    # Save to csv file
    df.to_csv(f"{html.split('_')[0]}.csv")

    # Append df to df list
    df_names.append((html, df))

# Print the DataFrames
for filename, df in df_names:
    print(f"DataFrame from file: {filename}")
    print(df)
    print()


C:\Users\rotem\AppData\Local\Temp/ipykernel_33252/3935485040.py:36: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Average_Price_in_Eur'] = pd.to_numeric(df['Average_Price_in_Eur'].str.replace('[^\d.]', ''), errors='coerce')


DataFrame from file: Paphos_numbeo.html
                                                 Item  Average_Price_in_Eur  \
0                        Meal, Inexpensive Restaurant                 12.00   
1   Meal for 2 People, Mid-range Restaurant, Three...                 45.00   
2      McMeal at McDonalds (or Equivalent Combo Meal)                  7.00   
3                   Domestic Beer (0.5 liter draught)                  2.80   
4                   Imported Beer (0.33 liter bottle)                  3.00   
5                                Cappuccino (regular)                  3.19   
6                      Coke/Pepsi (0.33 liter bottle)                  1.88   
7                           Water (0.33 liter bottle)                  0.81   
9                           Milk (regular), (1 liter)                  1.54   
10                   Loaf of Fresh White Bread (500g)                  1.35   
11                                Rice (white), (1kg)                  1.85   
12          